# Taller 2 — Sistema de Fidelización (POO + SQLite)

**Objetivo:** Diseñar y programar un sistema de puntos y recompensas aplicando **Herencia** y **Polimorfismo**, además de **persistencia** con SQLite. Este notebook está pensado para ser **clonado desde GitHub y ejecutado en cualquier PC** (macOS/Windows/Linux) sin tocar rutas manualmente.

## Guía del taller (adaptada y operativa)
- Implementar clases de dominio con **herencia**: `Cliente` (base) y `ClienteBronce`, `ClientePlata`, `ClienteOro` (subclases).
- Implementar **polimorfismo real**: cada subclase **sobrescribe** la forma de calcular puntos via multiplicadores.
- Reglas base: 1 punto por cada **$1.000 COP** (entero hacia abajo). Bono por **tarjeta aliada** (duplicación 2×).
- **Autoupgrade** de nivel por puntos acumulados: Plata ≥ 500; Oro ≥ 1.500. (No hay *downgrade* al redimir en este modelo).
- **Persistencia** en SQLite: tablas `clientes`, `transacciones`, `recompensas`. Con *seeding* de recompensas.
- Demostración end-to-end en este notebook (sin `input()`): alta de cliente, compras, redención, historial.

> **Nota:** Los comentarios en el código son del tipo que escribiría un humano con conocimiento **intermedio**, explicando decisiones de diseño sin sobrecomentar obviedades.

## Diagrama de clases (Mermaid)

```mermaid
classDiagram
  class Cliente {
    +id: int
    +nombre: str
    +email: str
    +puntos: int
    +nivel: str
    +fecha_registro: str
    +calcular_puntos(monto, tarjeta) int
    +aplicar_upgrade_si_corresponde() str
    +beneficios() list~str~
    -_multiplicador_tier() float
  }
  class ClienteBronce
  class ClientePlata
  class ClienteOro
  class LoyaltyEngine {
    +registrar_cliente()
    +registrar_compra()
    +redimir()
    +ver_cliente()
    +listar_recompensas()
    +historial()
  }
  class ClienteRepo
  class TransaccionRepo
  class RecompensaRepo

  Cliente <|-- ClienteBronce
  Cliente <|-- ClientePlata
  Cliente <|-- ClienteOro
  LoyaltyEngine --> ClienteRepo
  LoyaltyEngine --> TransaccionRepo
  LoyaltyEngine --> RecompensaRepo
```


## 1) Resolución **robusta** de la ruta de la base de datos

Estrategia de persistencia **portable**:
1. Si existe `FIDELIZABOT_DB`, usar esa ruta (permite override en laboratorios/servidores).
2. Usar `./data/fidelizabot.db` dentro del repo si es escribible.
3. Usar carpeta de datos del usuario (APPDATA/Library/`~/.local/share`).
4. Usar CWD como último intento.
5. Si nada es escribible (raro), usar `:memory:` (no persistente) y avisar.


In [ ]:
from __future__ import annotations
import os, sys, sqlite3
from dataclasses import dataclass
from datetime import datetime
from typing import Optional, List, Tuple, Type

def _is_writable_dir(path: str) -> bool:
    """True si `path` se puede crear/usar para escribir (probamos con un archivo temporal)."""
    try:
        os.makedirs(path, exist_ok=True)
        testfile = os.path.join(path, ".wtest")
        with open(testfile, "w", encoding="utf-8") as f:
            f.write("ok")
        os.remove(testfile)
        return True
    except Exception:
        return False

def _user_data_dir(app_name: str = "FidelizaBot") -> str:
    """Carpeta de datos por plataforma (sin dependencias externas)."""
    home = os.path.expanduser("~")
    if os.name == "nt":  # Windows
        base = os.environ.get("APPDATA", os.path.join(home, "AppData", "Roaming"))
        return os.path.join(base, app_name)
    elif sys.platform == "darwin":  # macOS
        return os.path.join(home, "Library", "Application Support", app_name)
    else:  # Linux/Unix
        return os.path.join(home, ".local", "share", app_name)

def resolve_db_path(filename: str = "fidelizabot.db") -> str:
    """Determina una ruta de DB segura y multiplataforma."""
    # 1) Variable de entorno
    env_path = os.environ.get("FIDELIZABOT_DB")
    if env_path:
        try:
            db_dir = os.path.dirname(env_path) or "."
            os.makedirs(db_dir, exist_ok=True)
            with open(os.path.join(db_dir, ".wtest"), "w", encoding="utf-8") as f:
                f.write("ok")
            os.remove(os.path.join(db_dir, ".wtest"))
            return env_path
        except Exception:
            print("[WARN] FIDELIZABOT_DB no es escribible. Intentando rutas alternativas...", file=sys.stderr)

    # 2) ./data dentro del proyecto
    project_dir = os.getcwd()
    data_dir = os.path.join(project_dir, "data")
    if _is_writable_dir(data_dir):
        return os.path.join(data_dir, filename)

    # 3) Carpeta de datos del usuario
    udir = _user_data_dir()
    if _is_writable_dir(udir):
        return os.path.join(udir, filename)

    # 4) CWD como último intento
    if _is_writable_dir(os.getcwd()):
        return os.path.join(os.getcwd(), filename)

    # 5) Fallback en memoria
    print("[WARN] Sin carpetas escribibles. Usando DB en memoria (no persiste).", file=sys.stderr)
    return ":memory:"

DB_PATH = resolve_db_path()

def get_conn() -> sqlite3.Connection:
    """Conexión SQLite con acceso a columnas por nombre (row_factory)."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

def init_db(seed: bool = True) -> None:
    """Crea el esquema si no existe y hace seeding de recompensas."""
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS clientes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL,
                email TEXT UNIQUE,
                puntos INTEGER NOT NULL DEFAULT 0,
                nivel TEXT NOT NULL CHECK(nivel IN ('BRONCE','PLATA','ORO')),
                fecha_registro TEXT NOT NULL
            );
            """
        )
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS transacciones (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                cliente_id INTEGER NOT NULL,
                fecha TEXT NOT NULL,
                monto_cop INTEGER NOT NULL,
                tarjeta_aliada INTEGER NOT NULL DEFAULT 0,
                puntos_ganados INTEGER NOT NULL DEFAULT 0,
                puntos_redimidos INTEGER NOT NULL DEFAULT 0,
                descripcion TEXT,
                FOREIGN KEY (cliente_id) REFERENCES clientes(id)
            );
            """
        )
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS recompensas (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL,
                costo_puntos INTEGER NOT NULL,
                descripcion TEXT
            );
            """
        )
        if seed:
            cur.execute("SELECT COUNT(*) AS c FROM recompensas;")
            if int(cur.fetchone()["c"]) == 0:
                cur.executemany(
                    "INSERT INTO recompensas (nombre, costo_puntos, descripcion) VALUES (?, ?, ?);",
                    [
                        ("Café pequeño", 120, "Café filtrado tamaño pequeño."),
                        ("Capuchino mediano", 250, "Capuchino en vaso mediano."),
                        ("Sandwich del día", 400, "Sandwich simple de la casa."),
                        ("Combo café + postre", 550, "Café y postre a elección."),
                        ("Merch oficial", 900, "Taza/termo/bolsa de la marca."),
                    ],
                )
        conn.commit()

init_db(seed=True)
DB_PATH

## 2) Dominio — Clases con **herencia** y **polimorfismo**

- `Cliente` define el contrato y lógica común.
- Subclases redefinen `_multiplicador_tier()` y pueden extender `beneficios()`.
- `calcular_puntos()` usa el multiplicador **sin conocer** la subclase concreta.

In [ ]:
@dataclass
class Cliente:
    id: Optional[int]
    nombre: str
    email: Optional[str]
    puntos: int
    nivel: str  # 'BRONCE' | 'PLATA' | 'ORO'
    fecha_registro: str

    # Reglas de negocio parametrizadas (fáciles de ajustar en una rúbrica posterior)
    BASE_POR_MIL: int = 1
    MULT_TARJETA: float = 2.0
    UMBRAL_PLATA: int = 500
    UMBRAL_ORO: int = 1500

    def calcular_puntos(self, monto_cop: int, tarjeta_aliada: bool) -> int:
        # Sanitización mínima; evita sumar puntos en montos inválidos
        if monto_cop <= 0:
            return 0
        base = (monto_cop // 1000) * self.BASE_POR_MIL
        mult = self._multiplicador_tier()
        if tarjeta_aliada:
            mult *= self.MULT_TARJETA
        return int(base * mult)

    def _multiplicador_tier(self) -> float:
        # Por defecto no hay premio adicional; las subclases lo ajustan
        return 1.0

    def aplicar_upgrade_si_corresponde(self) -> str:
        # Upgrade automático por puntos acumulados
        if self.puntos >= self.UMBRAL_ORO:
            return "ORO"
        if self.puntos >= self.UMBRAL_PLATA:
            return "PLATA"
        return "BRONCE"

    def beneficios(self) -> List[str]:
        return [
            "1 punto por cada $1.000 COP.",
            "Duplica puntos pagando con tarjeta aliada.",
        ]

class ClienteBronce(Cliente):
    def _multiplicador_tier(self) -> float:
        return 1.0
    def beneficios(self) -> List[str]:
        return super().beneficios() + ["Nivel Bronce: tasa base."]

class ClientePlata(Cliente):
    def _multiplicador_tier(self) -> float:
        return 1.25
    def beneficios(self) -> List[str]:
        return super().beneficios() + ["Nivel Plata: +25% puntos."]

class ClienteOro(Cliente):
    def _multiplicador_tier(self) -> float:
        return 1.5
    def beneficios(self) -> List[str]:
        return super().beneficios() + ["Nivel Oro: +50% puntos y prioridad."]

NIVEL_A_CLASE: dict[str, Type[Cliente]] = {
    "BRONCE": ClienteBronce,
    "PLATA": ClientePlata,
    "ORO": ClienteOro,
}

def cliente_from_row(row: sqlite3.Row) -> Cliente:
    cls = NIVEL_A_CLASE.get(row["nivel"], ClienteBronce)
    return cls(
        id=row["id"], nombre=row["nombre"], email=row["email"],
        puntos=row["puntos"], nivel=row["nivel"], fecha_registro=row["fecha_registro"],
    )

## 3) Repositorios — Acceso a datos sin ORM

Se implementan **funciones CRUD mínimas** por tabla para mantener el ejemplo legible y directo.

In [ ]:
class ClienteRepo:
    @staticmethod
    def crear(nombre: str, email: Optional[str]) -> Cliente:
        fecha = datetime.now().isoformat(timespec="seconds")
        with get_conn() as conn:
            cur = conn.cursor()
            cur.execute(
                "INSERT INTO clientes (nombre, email, puntos, nivel, fecha_registro) VALUES (?, ?, 0, 'BRONCE', ?);",
                (nombre.strip(), email, fecha),
            )
            new_id = cur.lastrowid
            cur.execute("SELECT * FROM clientes WHERE id = ?;", (new_id,))
            return cliente_from_row(cur.fetchone())

    @staticmethod
    def obtener(cliente_id: int) -> Optional[Cliente]:
        with get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT * FROM clientes WHERE id = ?;", (cliente_id,))
            row = cur.fetchone()
            return cliente_from_row(row) if row else None

    @staticmethod
    def actualizar(cliente_id: int, puntos: int, nivel: str) -> None:
        with get_conn() as conn:
            conn.execute("UPDATE clientes SET puntos = ?, nivel = ? WHERE id = ?;", (puntos, nivel, cliente_id))

class TransaccionRepo:
    @staticmethod
    def registrar(cliente_id: int, monto_cop: int, tarjeta_aliada: bool, puntos_g: int, puntos_r: int, desc: str) -> None:
        fecha = datetime.now().isoformat(timespec="seconds")
        with get_conn() as conn:
            conn.execute(
                """
                INSERT INTO transacciones (cliente_id, fecha, monto_cop, tarjeta_aliada, puntos_ganados, puntos_redimidos, descripcion)
                VALUES (?, ?, ?, ?, ?, ?, ?);
                """,
                (cliente_id, fecha, monto_cop, int(tarjeta_aliada), puntos_g, puntos_r, desc),
            )

    @staticmethod
    def historial(cliente_id: int) -> List[sqlite3.Row]:
        with get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT * FROM transacciones WHERE cliente_id = ? ORDER BY id DESC;", (cliente_id,))
            return cur.fetchall()

class RecompensaRepo:
    @staticmethod
    def listar() -> List[sqlite3.Row]:
        with get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT * FROM recompensas ORDER BY costo_puntos ASC;")
            return cur.fetchall()

    @staticmethod
    def obtener(recompensa_id: int) -> Optional[sqlite3.Row]:
        with get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT * FROM recompensas WHERE id = ?;", (recompensa_id,))
            return cur.fetchone()

## 4) Servicio de aplicación — **Casos de uso**

`LoyaltyEngine` orquesta reglas y persistencia. Aquí **se evidencia el polimorfismo** al calcular puntos sin conocer el tipo concreto del cliente.

In [ ]:
class LoyaltyEngine:
    def registrar_cliente(self, nombre: str, email: Optional[str]) -> Cliente:
        if not nombre or not nombre.strip():
            raise ValueError("El nombre es obligatorio.")
        return ClienteRepo.crear(nombre, email or None)

    def registrar_compra(self, cliente_id: int, monto_cop: int, tarjeta_aliada: bool, descripcion: Optional[str] = None) -> Tuple[Cliente, int]:
        c = ClienteRepo.obtener(cliente_id)
        if not c:
            raise ValueError("Cliente no encontrado.")
        if monto_cop <= 0:
            raise ValueError("El monto debe ser positivo.")

        puntos = c.calcular_puntos(monto_cop, tarjeta_aliada)
        TransaccionRepo.registrar(c.id, monto_cop, tarjeta_aliada, puntos, 0, descripcion or "Compra en tienda")
        c.puntos += puntos
        c.nivel = c.aplicar_upgrade_si_corresponde()
        ClienteRepo.actualizar(c.id, c.puntos, c.nivel)
        return ClienteRepo.obtener(c.id), puntos

    def redimir(self, cliente_id: int, recompensa_id: int) -> Tuple[Cliente, sqlite3.Row]:
        c = ClienteRepo.obtener(cliente_id)
        if not c:
            raise ValueError("Cliente no encontrado.")
        r = RecompensaRepo.obtener(recompensa_id)
        if not r:
            raise ValueError("Recompensa no encontrada.")
        costo = int(r["costo_puntos"])
        if c.puntos < costo:
            raise ValueError("Puntos insuficientes.")

        TransaccionRepo.registrar(c.id, 0, False, 0, costo, f"Redención: {r['nombre']}")
        c.puntos -= costo
        ClienteRepo.actualizar(c.id, c.puntos, c.nivel)  # En este modelo no hay downgrade
        return ClienteRepo.obtener(c.id), r

    def ver_cliente(self, cliente_id: int) -> Cliente:
        c = ClienteRepo.obtener(cliente_id)
        if not c:
            raise ValueError("Cliente no encontrado.")
        return c

    def listar_recompensas(self) -> List[sqlite3.Row]:
        return RecompensaRepo.listar()

    def historial(self, cliente_id: int) -> List[sqlite3.Row]:
        return TransaccionRepo.historial(cliente_id)

## 5) Demostración **end-to-end** (sin `input()`)

Simulamos un flujo típico: crear cliente → registrar varias compras con/sin tarjeta → listar recompensas → redimir → ver historial.

> Este bloque se puede **re-ejecutar** sin romper la base: añadirá más transacciones a ese cliente.

In [ ]:
engine = LoyaltyEngine()

# 1) Alta de cliente
cliente = engine.registrar_cliente("Ana Café", "ana@example.com")
print(f"Cliente creado: #{cliente.id} — {cliente.nombre} — Nivel {cliente.nivel} — {cliente.puntos} pts")

# 2) Compras (monto, tarjeta_aliada)
for monto, tarj in [(18000, False), (45500, True), (79999, False), (120000, True)]:
    cliente, pts = engine.registrar_compra(cliente.id, monto, tarj)
    print(f"Compra ${monto:,.0f} | tarjeta={'Sí' if tarj else 'No'} => +{pts} pts | Total={cliente.puntos} | Nivel={cliente.nivel}")

# 3) Mostrar recompensas
recs = engine.listar_recompensas()
print("\nRecompensas disponibles:")
for r in recs:
    print(f"  {r['id']}) {r['nombre']} — {r['costo_puntos']} pts")

# 4) Intento de redención (si alcanza para una recompensa intermedia)
objetivo = next((r for r in recs if r["costo_puntos"] <= cliente.puntos and r["costo_puntos"] >= 250), None)
if objetivo:
    cliente, r = engine.redimir(cliente.id, objetivo["id"])
    print(f"\nRedimido: {r['nombre']} ({r['costo_puntos']} pts) => Saldo ahora: {cliente.puntos} pts")
else:
    print("\nNo hay recompensas que pueda redimir aún.")

# 5) Historial
hist = engine.historial(cliente.id)
print("\nHistorial de transacciones:")
for h in hist:
    print(
        f"  #{h['id']} | {h['fecha']} | monto=${h['monto_cop']:,.0f} | tarjeta={'Sí' if h['tarjeta_aliada'] else 'No'} | +{h['puntos_ganados']} | -{h['puntos_redimidos']} | {h['descripcion'] or ''}"
    )

## 6) Consideraciones de diseño (para la rúbrica)

- **Herencia + Polimorfismo**: `calcular_puntos()` invoca `_multiplicador_tier()` sin conocer la subclase concreta.
- **Acoplamiento bajo**: lógica de dominio separada de persistencia (repos) y de orquestación (servicio).
- **Portabilidad**: resolución de DB tolerante a permisos (evita `OperationalError: unable to open database file`).
- **Extensibilidad**: fácil añadir categorías de producto, promociones por día, *downgrade* por inactividad, etc.
- **Pruebas**: el flujo E2E del bloque anterior puede convertirse en *tests* unitarios (p.ej., `pytest`).